# Основы веб-скрапинга и работы с API. ДЗ в рамках курса Нетологии "Data Scientist: с нуля до middle"¶

# Задание 1.

Будем парсить страницу со свежими новостями на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. 
Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [139]:
  
import requests

import datetime

import time

from bs4 import BeautifulSoup

import pandas as pd

# определяем список хабов, которые нам интересны
KEYWORDS = ['python', 'парсинг', 'api']  

habr_posts = pd.DataFrame()

# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
time.sleep(0.2)
soup = BeautifulSoup(req.text, 'html.parser')

# извлекаем посты
posts = soup.find_all('article', class_='tm-articles-list__item')

for post in posts:
    post_id = post.attrs.get('id')
    if not post_id: #на случай если у поста нет id
        continue

    #извлекаем ключевые слова из заголовков и тегов    
    hubs = post.select('a.tm-article-snippet__hubs-item-link, a.tm-article-snippet__title-link, \
                       div.tm-article-body tm-article-snippet__lead')
     
      
    for hub in hubs:
            hub_lower = hub.text.lower()

               
            # ищем вхождение хотя бы одного требуемого ключевого слова        
            if any([hub_lower in keyword for keyword in KEYWORDS]):
                

            #извлекаем дату в заданном формате
                date = pd.to_datetime(post.find('time').attrs.get('datetime'))

            #заголовок
                title_element = post.find('a', class_='tm-article-snippet__title-link')

            #ссылка на статью
                link = title_element.attrs.get('href')
                
            #формируем словарь структуры <дата> - <заголовок> - <ссылка>
                row = ({'date': datetime.datetime.strftime(date,'%d %B %Y %H:%M:%S'), 
                    'title': title_element.text, 'link': 'https://habr.com' + link})
            
            #вывод полного заголовка
                pd.set_option('display.max_colwidth', None)      

                habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])         
    

habr_posts.reset_index()


,index,date,title,link
0,0,24 July 2021 13:53:31,Автоматизация учёта в 1С,https://habr.com/ru/post/569424/
1,0,24 July 2021 08:54:29,Python кодогенерация на благо ETL,https://habr.com/ru/post/569398/


# Дополнительная часть (необязательная)

'''Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.


Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>'''

In [5]:


import re
URL = 'https://habr.com/ru/all/'
KEYWORDS_ = [ 'python','ассемблер', 'парсинг']

def get_all_links(url):
   
    """ Эта функция собирает ссылки на статьи на странице сайта """
    
    all_refs = []
   
    req = requests.get(URL)
    time.sleep(0.3)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    posts = soup.find_all('article', class_='tm-articles-list__item')
    posts_with_id = []
    for post in posts:
        post_id = post.attrs.get('id')
        if not post_id: #на случай если у поста нет id
            continue
        else:
            posts_with_id.append(post)
            
            
    articles_intro = list(map(lambda x: x.find('a', class_='tm-article-snippet__title-link'), posts_with_id))
    a_list = list(map(lambda x: x.attrs.get('href'), articles_intro))
    all_refs = list(map(lambda x: 'https://habr.com' + x, a_list))

    return all_refs 

all_links = get_all_links(URL)
all_links


['https://habr.com/ru/company/selectel/blog/568862/',
 'https://habr.com/ru/post/569392/',
 'https://habr.com/ru/post/569376/',
 'https://habr.com/ru/company/neuronet/blog/569374/',
 'https://habr.com/ru/company/audiomania/blog/568126/',
 'https://habr.com/ru/post/569360/',
 'https://habr.com/ru/post/569352/',
 'https://habr.com/ru/company/click/blog/569348/',
 'https://habr.com/ru/post/569346/',
 'https://habr.com/ru/post/569342/',
 'https://habr.com/ru/company/luxoft/blog/569330/',
 'https://habr.com/ru/company/2035_university/blog/569336/',
 'https://habr.com/ru/company/joom/blog/566304/',
 'https://habr.com/ru/post/569326/',
 'https://habr.com/ru/company/testit-tms/blog/568984/',
 'https://habr.com/ru/company/madrobots/blog/569312/',
 'https://habr.com/ru/post/569324/',
 'https://habr.com/ru/post/569318/',
 'https://habr.com/ru/company/ruvds/blog/567334/',
 'https://habr.com/ru/company/accenture/blog/569146/']

In [13]:
#ищем на страницах статей нужные слова
habr_posts_body = pd.DataFrame()

for link in all_links:

    soup_ = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    article = soup_.find('div', class_= 'tm-page-article__body')
    for keyword_ in KEYWORDS_:
        if re.search(keyword_, article.text, re.I):
            date_ = pd.to_datetime(soup_.find('time').attrs.get('datetime'))
            
            
            title_ = soup_.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1')
            
            article_text = article.find('div', class_ ='tm-article-body')
            
            
            #формируем словарь структуры <дата> - <заголовок> - <ссылка> - <текст_статьи>
            row_ = ({'date': datetime.datetime.strftime(date_,'%d %B %Y %H:%M:%S'), 
                    'title': title_.text, 'link': link, 'article_text': article_text.text})
            
            
            
            habr_posts_body = pd.concat([habr_posts_body, pd.DataFrame([row_])])
        

habr_posts_body.reset_index()
            

    

index                   date                                                                       title                                             link                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# Задание 2.
'''Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. 
Список email-ов задаем переменной в начале кода: 
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.'''

In [86]:

import requests
import json  

import requests

import datetime

import time

import pandas as pd
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'  
EMAIL = ['xxx@x.ru', 'yyy@ya.com']
param_dict = { 'emailAddresses': EMAIL }
headers = ({ 
        'Accept': 'application/json, text/plain, */*',
        'Accept-Encoding': 'gzip, deflate, br', 
        'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
        'Connection': 'keep-alive', 'Content-Type': 'application/json;charset=utf-8',
        'Host' : 'identityprotection.avast.com', 'Origin': 'https://www.avast.com', 
        'Referer': 'https://www.avast.com/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'Vaar-Header-App-Build-Version': '1.0.0', 'Vaar-Header-App-Product': 'hackcheck-web-avast',
        'Vaar-Header-App-Product-Name' : 'hackcheck-web-avast', 'Vaar-Version':'0'
           
                     
           })


response = requests.post(url, json = param_dict, headers = headers )
 

df = pd.DataFrame()

#фиксируем ответ от сервера
data = response.json()

#формируем датафрейм со столбцами: <email> - <дата утечки> - <источник утечки> - <описание утечки>
for email, b_id in data['summary'].items():
    for breachId in b_id['breaches']:
        for breach, value in data['breaches'].items():
            if breachId == int(breach):
                publishdate = pd.to_datetime(value['publishDate']) 
                site = value['site']
                description = value['description']
                dict_={'email': email, 'date':datetime.datetime.strftime(publishdate,'%d %B %Y %H:%M:%S'),'site':site, 'description':description}
                df = pd.concat([df, pd.DataFrame([dict_])])
    
df.reset_index()

    

    


,index,email,date,site,description
0,0,xxx@x.ru,21 October 2016 00:00:00,adobe.com,"In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software products. The affected products included Adobe's ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearly three-million customer credit card records and stole login data for an undisclosed number of Adobe user accounts."
1,0,xxx@x.ru,29 October 2016 00:00:00,vk.com,"Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a price of one bitcoin."
2,0,xxx@x.ru,23 October 2016 00:00:00,imesh.com,"In June 2016, a cache of over 51 million user credentials from the online sharing service iMesh appeared for sale on a dark web marketplace. The database contained user email addresses, usernames, passwords, IP addresses and location."
3,0,xxx@x.ru,31 January 2017 00:00:00,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum database was breached. The attacker may have exploited a vulnerability in vBulletin. The stolen data contains over 1,870,000 user records including associated usernames, email addresses, hashed passwords, and salts. The leaked credentials are being shared and sold privately on the dark web."
4,0,xxx@x.ru,14 February 2017 00:00:00,cfire.mail.ru,"In July and August of 2016, two criminals carried out attacks on three separate forums hosted by Mail.ru, including CFire. The hackers used known SQL injection vulnerabilities found in older vBulletin forum software to obtain access to the databases. Shortly after the breach itself, the contents of CFire's database were leaked publicly. The database contains usernames, email addresses, and MD5 hashed passwords for just under 13 million users."
5,0,xxx@x.ru,14 February 2017 00:00:00,parapa.mail.ru,"In July and August 2016, two criminals executed attacks against three separate forums hosted by Mail.ru including the Russian forum Parapa. Shortly after the breach occurred, the contents of Parapa's database were leaked publicly. The database contains usernames, email addresses, and hashed passwords for around 5 million users."
6,0,yyy@ya.com,04 April 2017 00:00:00,evony.com,"In July 2016, gaming site Evony's user database was breached. Over 33 million username and passwords were compromised including a number of associated Facebook login credentials. The breach became public in October 2016. Evony's user forum was also breached in August 2016. That seperate breach exposed an additional 938,000 user accounts."


# Дополнительная часть (необязательная)

'''Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>'''

In [91]:

from datetime import datetime

NETOLOGY_NEWS = 'https://api.vk.com/method/wall.get?'
TOKEN = ''
GROUP = 'netology'
VERSION = '5.131'
SLEEP = 0.33
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': GROUP,
    'count': 50
}
res = requests.get(NETOLOGY_NEWS, params)

news = pd.DataFrame(res.json()['response']['items'])

#преобразуем дату из utc
news['date'] = news['date'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

news

id   from_id  owner_id                 date  marked_as_ads post_type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        text  is_pinned                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     